In [18]:
import sys, os

# For .py
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
# For .ipynb
sys.path.append(os.path.dirname(os.getcwd()))

import json
import random
from util import Image_Loader
from util import find_random_question
from util import random_word
from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
import re
import ast

def remove_keyword_in_phrase(phrase, keyword):
    word_list = phrase.split(" ")
    preprocessed_phrase = " ".join([word for word in word_list if keyword not in word])
    return preprocessed_phrase

def remove_duplicate_word_from_phrases(phrase_list):
    token_list = []
    for phrase in phrase_list:
        token_list += [token.strip() for token in phrase.split(" ")]
    token_list = list(set(token_list))

    duplicated_tokens = []
    for token in token_list:
        if token in phrase_list[0] and token in phrase_list[1]:
            duplicated_tokens.append(token)
    preprocessed_phrase_list = []
    for duplicated_token in duplicated_tokens:
        for phrase in phrase_list:
            preprocessed_phrase_list.append(phrase.replace(duplicated_token, "").strip())
    return preprocessed_phrase_list

In [19]:
default_model="gpt-4o-mini"
verbose=True

total_cost = 0
keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_2'
random_questions = find_random_question(keyword, problem_type, detail_type=1, count=1, api_key_path="./../API_KEY/topik_api_key.json")
display(random_questions)

# Extract text between <ins> and </ins>
matches = re.findall(r'<ins>(.*?)</ins>', random_questions[0]['example'])
assert len(matches) == 1
underlined_text = matches[0].strip()
### Prepare a pair of sample phrases
a_pair_of_similar_phrases = [underlined_text, random_questions[0]['answer']]
display(a_pair_of_similar_phrases)

### Create a model for generating a problem
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.5, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

### get random word
main_topic = random_word()
if verbose:
    print(main_topic)

## Generate a new word based on the random word
problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 명사 1개를 제시하라. no intro. no outro."),
        HumanMessage(content="주제어: " + main_topic + " 명사: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
main_topic_generated = word_responses[0][1]
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))


[{'example': ' 어려운 이웃을 <ins>돕고자</ins> 매년 봉사 활동에 참여하고 있다.',
  'selector': ['돕기 위해서', '돕는 대신에', '돕기 무섭게', '돕는 바람에'],
  'answer': '돕기 위해서'}]

['돕고자', '돕기 위해서']

대낮
gpt-4o-mini-2024-07-18 | 
햇빛

OpenAI Tokens Used: 51
    Prompt Tokens: 48
    Completion Tokens: 3
Successful Requests: 1
Total Cost (USD): $0.0001
Total Cost (KRW): ₩0.135
--------------------------------------------------------------------------------------------------


- 유사 구문 생성

In [20]:
# example_phrases = ['[들릴만큼, 들릴정도로]', '[돕고자, 돕기위해서]', '[꾸미기 나름이다, 꾸미기에 달려있다]', '[타기만 하면, 탈 때마다]', '[수리해 봐야, 수리한다고 해도]', '[고향인 셈이다, 고향이나 마찬가지이다]']
# example_phrases = ['[들릴만큼, 들릴정도로]', '[돕고자, 돕기위해서]', '[타기만 하면, 탈 때마다]', '[수리해 봐야, 수리한다고 해도]']
# example_phrases = ['[들릴만큼, 들릴정도로]', '[타기만 하면, 탈 때마다]', '[수리해 봐야, 수리한다고 해도]']
# sample_phrases = random.sample(example_phrases, 1)
# a_pair_of_similar_phrases = sample_phrases[0]
human_prompt = f"두 구문은 서로 대체될 수 있어야 한다. 두 구문은 자연스러워야한다. 주제어: {main_topic_generated}"
display(human_prompt)

### Create a model for generating a phrase
models = ['gpt-4o']
phrase_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.1, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)
phrase_generate_model.request_models_responses(
    [
        SystemMessage(
            # content="#예시처럼 주어지는 주제어에 대한 문장을 제시하라. #답변 예시 "
            # + example_str
            content=f"너는 유사한 한국어 구문을 생성하는 봇이다. 예시 구문: {a_pair_of_similar_phrases}. No intro. No conclusion."
            # " #예시 문제와 같은 유형의 새로운 문제를 1개 만들어라. 문장 주제는 주제어를 참고하라." +
            # " #괄호는 관형구 또는 명사구로 구성된다"
        ),
        HumanMessage(content=human_prompt),
    ]
)

phrase_response = phrase_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', phrase_response[0][2]).group(1))

phrase_response = phrase_response[0][1]
phrase_response

# Step 1: Remove the unwanted characters
cleaned_str = phrase_response.replace("[", "").replace("]", "").replace("'", "").strip()

# Step 2: Split the string by the comma
elements = cleaned_str.split(",")

# Step 3: Strip any extra spaces
phrase_list = [element.strip() for element in elements]
phrase_list

# Remove any word that contains keyword
preprocessed_phrase_list = [remove_keyword_in_phrase(phrase, main_topic_generated) for phrase in phrase_list]
preprocessed_phrase_list
# # Use regular expression to find all items inside square brackets
# matches = re.findall(r'\[([^\]]+)\]', phrase_response)

# # Split the items within the matches to form a single list
# single_list = []
# for match in matches:
#     items = match.split(', ')
#     single_list.extend(items)

# generated_phrase = f"[{single_list[0]}, {single_list[1]}]"
# generated_phrase

'두 구문은 서로 대체될 수 있어야 한다. 두 구문은 자연스러워야한다. 주제어: 햇빛'

gpt-4o-2024-05-13 | 
['햇빛을 받기 위해서', '햇빛을 받으려고']

OpenAI Tokens Used: 101
    Prompt Tokens: 82
    Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.0002
Total Cost (KRW): ₩0.27
--------------------------------------------------------------------------------------------------


['받기 위해서', '받으려고']

- 문장 생성

In [21]:
human_prompt = f"""Make a sentence where either of the given phrases can be contained, but contain only one of them. Never change any letter in the phrase.
Phrases: [{preprocessed_phrase_list[0]}, {preprocessed_phrase_list[1]}]"""
# human_prompt = f"""Contain either of the given phrases. Don't change any letter in the phrase.
# Phrases: {preprocessed_phrase_list[0]}, {preprocessed_phrase_list[1]}"""
display(human_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content=f"#너는 자연스러운 한국어 문장을 생성하는 봇이다. 주제어: {main_topic_generated}"
        ),
        HumanMessage(content=human_prompt),
    ]
)

sentence_response = problem_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', sentence_response[0][2]).group(1))

sentence_response = sentence_response[0][1]
display(sentence_response)

target_phrase = None
similar_phrase = None
for phrase in preprocessed_phrase_list:
    if phrase in sentence_response:
        target_phrase = phrase
    else:
        similar_phrase = phrase
assert target_phrase is not None, "Target phrase not found in the sentence"
assert similar_phrase is not None, "Similar phrase not found in the sentence"
print(f'Target Phrase: {target_phrase}\nSimilar Phrase: {similar_phrase}')

'Make a sentence where either of the given phrases can be contained, but contain only one of them. Never change any letter in the phrase.\nPhrases: [받기 위해서, 받으려고]'

gpt-4o-mini-2024-07-18 | 
햇빛을 받기 위해서 나무 아래에서 책을 읽었다.

OpenAI Tokens Used: 92
    Prompt Tokens: 75
    Completion Tokens: 17
Successful Requests: 1
Total Cost (USD): $0.00018
Total Cost (KRW): ₩0.243
--------------------------------------------------------------------------------------------------


'햇빛을 받기 위해서 나무 아래에서 책을 읽었다.'

Target Phrase: 받기 위해서
Similar Phrase: 받으려고


- 선택지 생성

In [22]:
# example_sentence = "어려운 이웃을 돕고자 매년 봉사 활동에 참여하고 있다."
# example_question = {
#     '문장': example_sentence,
#     '밑줄' : '돕고자',
#     '정답': '돕기 위해서',
#     '오답': ['돕는 대신에', '돕기 무섭게', '돕는 바람에']
# }
### Make an example question for system message to refer to
random_question = random_questions[0]
example_sentence = random_question['example'].replace('<ins>', '').replace('</ins>', '')
example_question = {
    '문장': example_sentence,
    '밑줄' : underlined_text,
    '정답': random_question['answer'],
    '오답': [selector for selector in random_question['selector'] if selector != random_question['answer']]
}
### prompt
# human_prompt = f"""문장: {sentence_response}
# 밑줄: "{target_phrase}"
# 모든 선택지는 밑줄에 자연스럽게 들어갈 수 있어야한다. 선택지의 어간은 모두 동일해야한다."""
human_prompt = f"""문장: {sentence_response}
밑줄: "{target_phrase}"
모든 선택지의 어간은 동일해야하며, 밑줄에 자연스럽게 들어갈 수 있어야한다."""
display(human_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content=f"""너는 문장의 밑줄 친 부분과 의미가 가장 비슷한 선택지를 만드는 봇이다. 오답 3개, 정답 1개의 선택지를 만들어라.
            #답변 예시: {example_question}"""
        ),
        HumanMessage(content=human_prompt),
    ]
)

response = problem_generate_model.get_model_responses()

total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', response[0][2]).group(1))

selectors = response[0][1]
selectors = ast.literal_eval(selectors)
selectors = selectors['오답']
selectors.append(similar_phrase)
selectors

'문장: 햇빛을 받기 위해서 나무 아래에서 책을 읽었다.\n밑줄: "받기 위해서"\n모든 선택지의 어간은 동일해야하며, 밑줄에 자연스럽게 들어갈 수 있어야한다.'

gpt-4o-mini-2024-07-18 | 
{'문장': '햇빛을 받기 위해서 나무 아래에서 책을 읽었다.', '밑줄': '받기 위해서', '정답': '얻기 위해서', '오답': ['가리기 위해서', '피하기 위해서', '막기 위해서']}

OpenAI Tokens Used: 253
    Prompt Tokens: 186
    Completion Tokens: 67
Successful Requests: 1
Total Cost (USD): $0.00051
Total Cost (KRW): ₩0.6885
--------------------------------------------------------------------------------------------------


['가리기 위해서', '피하기 위해서', '막기 위해서', '받으려고']

In [23]:
final_sentence = sentence_response.replace(target_phrase, f"<ins>{target_phrase}</ins>")

In [24]:
result = {
    "example": final_sentence,
    "selector": selectors,
    "answer": similar_phrase,
    "eval_answer": similar_phrase,
    "total_cost": total_cost,
}
result

{'example': '햇빛을 <ins>받기 위해서</ins> 나무 아래에서 책을 읽었다.',
 'selector': ['가리기 위해서', '피하기 위해서', '막기 위해서', '받으려고'],
 'answer': '받으려고',
 'eval_answer': '받으려고',
 'total_cost': 1.3365}